In [7]:
import asyncio
import httpx

# Define the endpoint for the containerized API
API_URL = "http://localhost:8000/inference"  # Update the URL and endpoint as needed

# Example data to send in the POST request
PAYLOAD = {"input": "This is a test input for the model."}

# Number of parallel requests
NUM_REQUESTS = 10


async def send_request(client: httpx.AsyncClient, request_id: int):
    """Send a single POST request to the containerized API and print the response."""
    try:
        response = await client.post(API_URL, json=PAYLOAD)
        print(f"Request {request_id}: Status {response.status_code}, Response: {response.json()}")
    except Exception as e:
        print(f"Request {request_id}: Failed with error: {e}")


async def main():
    """Send multiple POST requests in parallel."""
    async with httpx.AsyncClient() as client:
        tasks = [send_request(client, i) for i in range(NUM_REQUESTS)]
        await asyncio.gather(*tasks)


if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import requests
import json

Test the local API

In [32]:
url = 'http://127.0.0.1:8000/infer'
data = {'input': 'I am happy with the new shoes i bought from '}

# Send POST request with JSON data
%time response = requests.post(url, data=json.dumps(data), headers={'Content-Type': 'application/json'})


# Print the response
print(response.json())

CPU times: user 10.9 ms, sys: 10.2 ms, total: 21.1 ms
Wall time: 230 ms
{'prediction': [{'label': 'POSITIVE', 'score': 0.9997774958610535}]}


In [35]:
response.text

'{"prediction":[{"label":"POSITIVE","score":0.9997774958610535}]}'

In [34]:
json.loads(response.text)['prediction']

[{'label': 'POSITIVE', 'score': 0.9997774958610535}]

Test the local docker api

In [6]:
url = 'http://0.0.0.0:8000/'
data = {'input': 'This is a positive input for the model.'}
# Send POST request with JSON data
#%time response = requests.post(url, data=json.dumps(data), headers={'Content-Type': 'application/json'})

response = requests.get(url)

# Print the response
print(response.json())

{'message': 'Model Inference API'}


AWS docker api

In [5]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Define the API endpoint
url = 'http://0.0.0.0:8000/infer'

# Sample texts for sentiment analysis
texts = [
    # Add a larger list of texts to better demonstrate concurrency
    "I love this product!",
    "This is the worst service ever.",
    "I'm feeling great today.",
    "I am not happy with the results.",
    "Could be better, but I'm satisfied.",
    "Absolutely fantastic experience!",
    "Not what I expected, quite disappointing.",
    "The service was acceptable.",
    "Exceeded all my expectations!",
    "I wouldn't recommend this to anyone.",
    # Add more texts if desired
]

def send_request(text):
    payload = {'text': text}
    try:
        print('here')
        response = requests.post(url, json=payload)
        print(response.text)
        if response.status_code == 200:
            result = response.json()
            print(f"Input Text: {text}")
            print(f"Predicted Sentiment: {result['sentiment']}")
            print(f"Confidence Score: {result['confidence']:.4f}\n")
        else:
            print(f"Request failed with status code {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

start_time = time.time()

# Number of threads to use
num_threads = 5

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(send_request, text) for text in texts]
    for future in as_completed(futures):
        pass

end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken: {total_time:.2f} seconds")


here
here
here
{"detail":[{"type":"missing","loc":["body","input"],"msg":"Field required","input":{"text":"I love this product!"}}]}
Request failed with status code 422
{"detail":[{"type":"missing","loc":["body","input"],"msg":"Field required","input":{"text":"This is the worst service ever."}}]}
Request failed with status code 422
here
{"detail":[{"type":"missing","loc":["body","input"],"msg":"Field required","input":{"text":"I'm feeling great today."}}]}
Request failed with status code 422
here
here
here
here
{"detail":[{"type":"missing","loc":["body","input"],"msg":"Field required","input":{"text":"I am not happy with the results."}}]}
Request failed with status code 422
here
{"detail":[{"type":"missing","loc":["body","input"],"msg":"Field required","input":{"text":"The service was acceptable."}}]}
Request failed with status code 422
here
{"detail":[{"type":"missing","loc":["body","input"],"msg":"Field required","input":{"text":"Could be better, but I'm satisfied."}}]}
Request faile

In [4]:
import requests
from concurrent.futures import ThreadPoolExecutor

url = "http://localhost:8000/infer"

payload = {"input": "I love this product!"}

# Function to make a POST request
def make_request():
    response = requests.post(url, json=payload)
    print(response.status_code, response.json())

# Send 10 parallel requests
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(make_request) for _ in range(10)]


200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}
200 {'prediction': [{'label': 'positive', 'score': 0.9848045110702515}]}


In [3]:
import requests
from concurrent.futures import ThreadPoolExecutor

# Define the API endpoint and input data
url = "http://0.0.0.0:8000/infer"
texts = [
    "I love this product!",
    "This is terrible.",
    "Not sure about this one.",
    "Could be better.",
    "Absolutely amazing experience!"
]

def send_request(text):
    response = requests.post(url, json={"text": text})
    if response.status_code == 200:
        print(response.json())
    else:
        print(f"Error: {response.status_code}")

# Send requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(send_request, texts)


Error: 422
Error: 422
Error: 422
Error: 422
Error: 422
